In [14]:
import pandas as pd
from pathlib import Path

In [21]:
ILUMINA_PATH = Path('../data/Ilumina')
ILUMINA_CLEAN_PATH = Path('../data/Ilumina/clean')

NANOPORE_PATH = Path('../data/Nanopore')
NANOPORE_CLEAN_PATH = Path('../data/Nanopore/clean')

WindowsPath('../data/Ilumina')

In [50]:
IluminaPylum = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Pylum-level microbiota")
IluminaFamily = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Family-level microbiota")
IluminaGenus = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Genus-level microbiota")

NanoporePylum = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Pylum-level microbiota")
NanoporeFamily = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Family-level microbiota")
NanoporeGenus = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Genus-level microbiota")

In [56]:
IluminaPylum["Fertil"] = 1
IluminaPylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaFamily["Fertil"] = 1
IluminaFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaGenus["Fertil"] = 1
IluminaGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporePylum["Fertil"] = 1
NanoporePylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeFamily["Fertil"] = 1
NanoporeFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeGenus["Fertil"] = 1
NanoporeGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

In [1]:
IluminaPylum.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylum.csv")

NameError: name 'IluminaPylum' is not defined